In [1]:
import os
import requests
from bs4 import BeautifulSoup
from typing import List
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai
from IPython.display import update_display, display, Markdown
import anthropic
import gradio as gr

In [2]:
load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

OpenAI API Key exists and begins sk-proj-
Anthropic API Key exists and begins sk-ant-
Google API Key exists and begins AIzaSyCp


In [3]:
openai = OpenAI()
claude = anthropic.Anthropic()
google.generativeai.configure()

In [4]:
system_prompt = "You are an assistant that analyzes the contents of a company website landing page \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown."

In [5]:
def stream_gpt(prompt):
    stream = openai.chat.completions.create(
        messages = [
            {'role':'system','content':system_prompt},
            {'role':'user','content':prompt}
        ],
        model = 'gpt-4o-mini',
        stream = True
    )
    response = ''
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace('```','').replace('markdown','')
        yield response

In [6]:
def stream_claude(prompt):
    response = claude.messages.stream(
        max_tokens = 5000,
        model ='claude-3-7-sonnet-20250219',
        system= system_prompt,
        messages = [{'role':'user','content':prompt}]
    )
    result =''
    with response as stream:
        for text in stream.text_stream:
            result += text or ''
            yield result

In [7]:
def stream_gemini(prompt):
    gemini = google.generativeai.GenerativeModel(
        model_name='gemini-2.5-flash-lite',
        system_instruction=system_prompt
    )

    result = gemini.generate_content(prompt, stream=True)
    response = ''
    for chunk in result:
        if chunk.text:               
            response += chunk.text
            yield response

# Web Scrapping to make a Brochure

In [8]:
def scape_webpage(myurl):
    '''Scrape any webpage easily using this function'''
    response = requests.get(myurl)
    soup = BeautifulSoup(response.content,'html.parser')

    title = soup.title.string if soup.title else "No Title found."

    for irrelevant in soup.body(['script','style','img','input']):
        irrelevant.decompose()
    text = soup.body.get_text(separator = '\n', strip = True)

    return f"Webpage Title:\n{title}\nWebpage Contents:\n{text}\n\n"

In [9]:
def stream_brochure(company_name,myurl,model):
    prompt = f"Please generate a company brochure for {company_name}. Here is their landing page:\n"
    prompt += scape_webpage(myurl)
    if model =='GPT':
        response = stream_gpt(prompt)
    elif model =='Claude':
        response = stream_claude(prompt)
    elif model =='Gemini':
        response = stream_gemini(prompt)
    else:
        raise ValueError('Model Not Found')

    yield from response

In [11]:
gr.Interface(
    fn = stream_brochure,
    inputs = [gr.Textbox(label = 'Company Name'),
              gr.Textbox(label= 'URL'),
              gr.Dropdown(['GPT','Claude','Gemini'],label = 'Select Your Model')
             ],
    outputs = [gr.Markdown('Brochure:')],
    allow_flagging = 'never'
).launch(share=True)

/opt/anaconda3/envs/llms/lib/python3.11/site-packages/gradio/interface.py:414: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated. Use `flagging_mode` instead.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://762f641535c40b8b54.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
